In [5]:
import boto3

In [6]:
import os
os.environ.setdefault('AWS_PROFILE','ghactivity')

'ghactivity'

In [7]:
def get_job_details(job_name):
    dynamodb=boto3.resource('dynamodb')
    table=dynamodb.Table('jobs')
    job_details=table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [9]:
job_details=get_job_details('ghactivity_ingest')
job_details

{'job_description': 'ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [10]:
from datetime import datetime as dt
from datetime import timedelta as td

In [12]:
start_file=next_file=f'{dt.strftime(dt.now().date() - td(days=6), "%Y-%m-%d")}-0.json.gz'

In [13]:
job_details={
    'job_id': 'ghactivity_ingest',
    'job_description': 'ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3,
    'job_run_bookmark_details':{
        'last_run_file_name': start_file
    }

}

In [30]:
dynamodb=boto3.resource('dynamodb')
job_details_table=dynamodb.Table('jobs')

In [31]:
job_details_table.put_item(
    Item=job_details)


{'ResponseMetadata': {'RequestId': 'G75HPG64T9I44H6HEPRPNMRHLRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 25 Jul 2022 15:24:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'G75HPG64T9I44H6HEPRPNMRHLRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [32]:
table=dynamodb.Table('jobs')
table.get_item(Key={'job_id':'ghactivity_ingest'})


{'Item': {'job_description': 'ingest ghactivity data to s3',
  'is_active': 'Y',
  'job_id': 'ghactivity_ingest',
  'baseline_days': Decimal('3'),
  'job_run_bookmark_details': {'last_run_file_name': '2022-07-19-0.json.gz'}},
 'ResponseMetadata': {'RequestId': '58J8TUN1R5I9O54JLGA2T16HLVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 25 Jul 2022 15:24:51 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '233',
   'connection': 'keep-alive',
   'x-amzn-requestid': '58J8TUN1R5I9O54JLGA2T16HLVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1867438545'},
  'RetryAttempts': 0}}

In [17]:
job_run_bookmark_details= table.get_item(Key={'job_id':'ghactivity_ingest'})['Item']['job_run_bookmark_details']

In [18]:
job_run_bookmark_details

{'last_run_file_name': '2022-07-19-0.json.gz'}

In [19]:
df_part=job_run_bookmark_details['last_run_file_name'].split('.')[0]

In [20]:
df_part

'2022-07-19-0'

In [21]:
next_file = f"{dt.strftime(dt.strptime(df_part,'%Y-%m-%d-%H')+ td(hours=1), '%Y-%m-%d-%-H')}.json.gz"

In [22]:
next_file

'2022-07-19-1.json.gz'

In [23]:
import requests

In [24]:
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [25]:
res

<Response [200]>

In [26]:
file  = open(f'data/{next_file}', 'wb')

In [27]:
file.write(res.content)

65308092

In [28]:
file.close()

In [29]:
!ls -ltr data/

total 466776
-rw-r--r--  1 shubhangigupta  staff  33989636 Jul 16 19:30 2022-06-05-0.json.gz
-rw-r--r--  1 shubhangigupta  staff  81204734 Jul 17 00:15 2022-07-14-0.json.gz
-rw-r--r--  1 shubhangigupta  staff  58475438 Jul 17 21:06 2022-07-14-1.json.gz
-rw-r--r--  1 shubhangigupta  staff  65308092 Jul 25 11:21 2022-07-19-1.json.gz


In [61]:
import pandas as pd

In [62]:
df = pd.read_json(f'data/{next_file}', lines=True, orient='record')

In [46]:
df

""


In [63]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')